In [1]:
import sys
sys.path.append("../..")
from thesis.utils.utils import setup_ray, save, load, Experiment
path = "D:/Master/Masterarbeit/thesis"
setup_ray(path = path, unidirectional = False, seed=69)

2023-01-14 11:03:50,304	INFO worker.py:1528 -- Started a local Ray instance.


In [2]:
env_args = dict(
    fleetsize = 2,
    max_fleetsize = 10,    
    pseudo_routing = False,
    pseudo_dispatcher = True,
    #pseudo_dispatcher_distance = 0.2,
    routing_agent_death= False,
    death_on_target = False,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 1,
        #reward_reached_target_by_time = True, 
        reward_wrong_target = -0.5,
        reward_removed_for_block = -1, 
        reward_target_distance = -0.05,
        reward_invalid= -0.1,
        block_timeout = 120,
        reward_accepted_in_station = 1,
        reward_declined_in_station = -1,
        routing_interval = 2,
        dispatching_interval=360,
        io_quote = 0.95  ,
        availability = 0.9,
        mttr = 5,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=64,
            with_action_mask=False,
            with_agvs=True,
            with_stations = False,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
            #env_type = "minimatrix",
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=32,
            with_action_mask=False,
            with_agvs=True,
            with_stations = True,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
        )
    )
)

In [4]:
exp = Experiment("minimatrix_routing")
for seed in [42,43,44]:
    exp.experiment(
        path = path,
        env_args = env_args, 
        agv_model = agv_model,
        dispatcher_model=dispatcher_model,
        run_name="11_reward_dist_", 
        env = "minimatrix",
        algo = "dqn",
        n_intervals =3,
        #train_agv = False,
        backup_interval=50,
        #batch_size=500, #apex + gnn: 50
        seed = seed,
        algo_params = {"gamma":0.98},
        lr = 1e-3,
        #load_agv="../../models/minimatrix_dispatching/04_complete__2_10_2023-01-13_12-09-48/checkpoint_000150/"
    )

2023-01-14 11:04:01,647	INFO simple_q.py:307 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2023-01-14 11:04:01,647	INFO algorithm.py:457 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_46113_ah6b63o8)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_46130_c89c6lya)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma

In [ ]:
exp.trainer.workers.local_worker().get_policy("agv").get_exploration_state()

{'cur_epsilon': 0.9868288, 'last_timestep': 1344}